In [6]:
import joblib
import numpy as np

import keras
from keras.models import save_model
from keras.models import Sequential
from keras.layers import LSTM, GRU, Conv1D, MaxPooling1D, Flatten, Dense, Input
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau

from scikeras.wrappers import KerasClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report


filePath = './data_train_valid.npz'
data = np.load(filePath)

X_train = data['X_train']
y_train = data['y_train']
X_valid = data['X_valid']
y_valid = data['y_valid']

In [7]:
def modelLST(X_train=X_train, y_train=y_train, X_valid=X_valid, y_valid=y_valid):
    model = Sequential([
        Input(shape=(X_train.shape[1], X_train.shape[2])),
        LSTM(units=128, return_sequences=True),
        LSTM(units=64),
        Dense(units=32, activation='relu'),
        Dense(units=1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, validation_data=(X_valid, y_valid))
    return model

def modelGRU(X_train=X_train, y_train=y_train, X_valid=X_valid, y_valid=y_valid):
    model = Sequential([
        Input(shape=(X_train.shape[1], X_train.shape[2])),
        GRU(units=128, return_sequences=True),
        GRU(units=64),
        Dense(units=32, activation='relu'),
        Dense(units=1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, validation_data=(X_valid, y_valid))
    return model

def modelTCN(X_train=X_train, y_train=y_train, X_valid=X_valid, y_valid=y_valid):
    model = Sequential([
        Input(shape=(X_train.shape[1], X_train.shape[2])),
        Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'),
        MaxPooling1D(pool_size=1),
        Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'),
        MaxPooling1D(pool_size=1),
        Flatten(),
        Dense(units=32, activation='relu'),
        Dense(units=1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.01, min_lr=0.0001)
    model.fit(X_train, y_train, validation_data=(X_valid, y_valid), callbacks=[reduce_lr])
    return model

In [8]:
def ensembleModel(epoch: int, weights: list, nameFiles: str):
    # Membuat objek KerasClassifier untuk masing-masing model
    lstm_classifier = KerasClassifier(model=modelLST, epochs=epoch, verbose=0)
    gru_classifier  = KerasClassifier(model=modelGRU, epochs=epoch, verbose=0)
    tcn_classifier  = KerasClassifier(model=modelTCN, epochs=epoch, verbose=0)

    # Definisikan model-model individu sebagai sebuah list
    models = [
        ('lst', lstm_classifier),
        ('gru', gru_classifier),
        ('tcn', tcn_classifier)
    ]

    # Membuat ensemble model dengan soft voting
    ensemble_model = VotingClassifier(estimators=models,
                                      voting='soft',
                                      weights=weights)

    # Melatih ensemble model
    ensemble_model.fit(X_train, y_train)

    # Melakukan prediksi menggunakan ensemble model
    y_pred_ensemble = ensemble_model.predict(X_valid)

    # Menampilkan classification report
    print("Ensemble Model Classification Report:")
    print(classification_report(y_valid, y_pred_ensemble))
    
    # Simpan model ensemble
    joblib.dump(ensemble_model, nameFiles + '.pkl')

## Train Model
### Epoch 10 and Weight 1:1:1

In [9]:
ensembleModel(10, [1, 1, 1], 'ep10_wgt1_1_1')

50/50 ━━━━━━━━━━━━━━━━━━━━ 13s 205ms/step - accuracy: 0.7578 - loss: 0.5819 - val_accuracy: 0.8350 - val_loss: 0.4922
50/50 ━━━━━━━━━━━━━━━━━━━━ 10s 139ms/step - accuracy: 0.7342 - loss: 0.5422 - val_accuracy: 0.8525 - val_loss: 0.4135
50/50 ━━━━━━━━━━━━━━━━━━━━ 33s 626ms/step - accuracy: 0.7306 - loss: 1.6340 - val_accuracy: 0.7875 - val_loss: 1.2847 - learning_rate: 0.0010
Ensemble Model Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.94      0.92       201
           1       0.93      0.90      0.92       199

    accuracy                           0.92       400
   macro avg       0.92      0.92      0.92       400
weighted avg       0.92      0.92      0.92       400



### Epoch 10 and Weight 1:1:2

In [10]:
ensembleModel(10, [1, 1, 2], 'ep10_wgt1_1_2')

50/50 ━━━━━━━━━━━━━━━━━━━━ 14s 223ms/step - accuracy: 0.7516 - loss: 0.5876 - val_accuracy: 0.8425 - val_loss: 0.4896
50/50 ━━━━━━━━━━━━━━━━━━━━ 11s 142ms/step - accuracy: 0.7579 - loss: 0.5058 - val_accuracy: 0.8625 - val_loss: 0.3973
50/50 ━━━━━━━━━━━━━━━━━━━━ 31s 595ms/step - accuracy: 0.7399 - loss: 1.0561 - val_accuracy: 0.8450 - val_loss: 0.5920 - learning_rate: 0.0010
Ensemble Model Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.95      0.91       201
           1       0.94      0.87      0.91       199

    accuracy                           0.91       400
   macro avg       0.91      0.91      0.91       400
weighted avg       0.91      0.91      0.91       400



### Epoch 20 and Weight 1:1:1

In [11]:
ensembleModel(20, [1, 1, 1], 'ep20_wgt1_1_1')

50/50 ━━━━━━━━━━━━━━━━━━━━ 12s 183ms/step - accuracy: 0.7581 - loss: 0.5811 - val_accuracy: 0.8500 - val_loss: 0.4781
50/50 ━━━━━━━━━━━━━━━━━━━━ 10s 137ms/step - accuracy: 0.7398 - loss: 0.5342 - val_accuracy: 0.8700 - val_loss: 0.3944
50/50 ━━━━━━━━━━━━━━━━━━━━ 32s 623ms/step - accuracy: 0.7228 - loss: 1.5858 - val_accuracy: 0.8300 - val_loss: 0.5299 - learning_rate: 0.0010
Ensemble Model Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.95      0.91       201
           1       0.94      0.87      0.90       199

    accuracy                           0.91       400
   macro avg       0.91      0.91      0.91       400
weighted avg       0.91      0.91      0.91       400



### Epoch 20 and Weight 1:1:2

In [12]:
ensembleModel(20, [1, 1, 2], 'ep20_wgt1_1_2')

50/50 ━━━━━━━━━━━━━━━━━━━━ 12s 187ms/step - accuracy: 0.7686 - loss: 0.5859 - val_accuracy: 0.8675 - val_loss: 0.4754
50/50 ━━━━━━━━━━━━━━━━━━━━ 12s 160ms/step - accuracy: 0.7391 - loss: 0.5097 - val_accuracy: 0.8650 - val_loss: 0.3876
50/50 ━━━━━━━━━━━━━━━━━━━━ 34s 651ms/step - accuracy: 0.7108 - loss: 1.1846 - val_accuracy: 0.8150 - val_loss: 0.4769 - learning_rate: 0.0010
Ensemble Model Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.88      0.90       201
           1       0.88      0.92      0.90       199

    accuracy                           0.90       400
   macro avg       0.90      0.90      0.90       400
weighted avg       0.90      0.90      0.90       400



### Epoch 30 and Weight 1:1:1

In [13]:
ensembleModel(30, [1, 1, 1], 'ep30_wgt1_1_1')

50/50 ━━━━━━━━━━━━━━━━━━━━ 13s 195ms/step - accuracy: 0.7506 - loss: 0.5962 - val_accuracy: 0.8450 - val_loss: 0.4840
50/50 ━━━━━━━━━━━━━━━━━━━━ 11s 150ms/step - accuracy: 0.7903 - loss: 0.4896 - val_accuracy: 0.8725 - val_loss: 0.3703
50/50 ━━━━━━━━━━━━━━━━━━━━ 33s 633ms/step - accuracy: 0.7341 - loss: 1.4125 - val_accuracy: 0.8050 - val_loss: 0.9067 - learning_rate: 0.0010
Ensemble Model Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.95      0.92       201
           1       0.94      0.90      0.92       199

    accuracy                           0.92       400
   macro avg       0.92      0.92      0.92       400
weighted avg       0.92      0.92      0.92       400



### Epoch 30 and Weight 1:1:2

In [14]:
ensembleModel(30, [1, 1, 2], 'ep30_wgt1_1_2')

50/50 ━━━━━━━━━━━━━━━━━━━━ 16s 194ms/step - accuracy: 0.7154 - loss: 0.5979 - val_accuracy: 0.8475 - val_loss: 0.4982
50/50 ━━━━━━━━━━━━━━━━━━━━ 11s 149ms/step - accuracy: 0.7381 - loss: 0.5259 - val_accuracy: 0.8825 - val_loss: 0.4059
50/50 ━━━━━━━━━━━━━━━━━━━━ 33s 635ms/step - accuracy: 0.7039 - loss: 1.8069 - val_accuracy: 0.8375 - val_loss: 0.9870 - learning_rate: 0.0010
Ensemble Model Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.95      0.93       201
           1       0.94      0.90      0.92       199

    accuracy                           0.93       400
   macro avg       0.93      0.92      0.92       400
weighted avg       0.93      0.93      0.92       400

